# One-Shot Document Classification

This jupyter notebook demonstrates the workflow used in the documentation for **Boon AI's Python SDK**. You can find that walk-through [here](https://app.gitbook.com/@zorroa/s/boonsdk/solutions/document-classification).<br>

## Setup: 

In [1]:
import boonsdk
app = boonsdk.app_from_keyfile("./apikey.json")
# In this case, the apikey.json was assigned that name from the web application and stored in the same directory
# as the notebook you are working on. Be sure to point to the location and name of your api key. 

env: BOONAI_SERVER=https://dev.api.boonai.app


## Dataset:
Now that we have set up the environment, we can define the dataset. We will be using a sample set comprised of two resumes and two cover letters to train our model. Remember, a dataset is a collection of assets to which a label has been assigned. In order for a model to be trained, it must be attached to a dataset. For more information on datasets, refer to datasets subheading, under custom models in our documentation. You can find it [here](https://app.gitbook.com/@zorroa/s/boonsdk/training-models/datasets).

In [2]:
# Hold onto this dataset, assigning it to an easy to reference variable, because we'll need it later on.
dataset = app.datasets.create_dataset(
    "job-documents-dataset", 
    boonsdk.DatasetType.Classification
)

 There is an images.csv file in this directory. It consists of two columns and five rows. The first row is a header. The first column points to the uri for an image and the second applies a label to that image. If you have downloaded this file, be sure to provide the path to your local version of "images.csv" By providing labels here we are cerating labels to be associated with the dataset. After running this next block of code, you will recieve an object enumerating errors, number of assets you have created, and number of assets that exist. If you see a number in 'exists', it means that you have attempted to upload a duplicate of a file already in your project. 

In [4]:
# If you have downloaded this file, be sure to provide the path to your local version of "images.csv"
path_to_csv = "./images.csv"

# The file we're importing has values separated by commas, which is the default delimeter. 
# Each uri and label is enclosed in double quotes. 
csvfile = boonsdk.CsvFileImport(path_to_csv,
                                uri_column=0,
                                dataset=dataset,
                                label_column=1,
                                max_assets=1000, 
                                header=True)

app.assets.import_csv(csvfile)

{'errors': 0, 'created': 0, 'exists': 4}

## Model:
Now that we have a dataset imported, we can create and train a model that will predict the type of any future documents we add to this project. We will use a KNN Classifier and attach the dataset we created earlier. 

In [5]:
model = app.models.create_model(
    "document-classifier",
    boonsdk.ModelType.KNN_CLASSIFIER,
    dataset=dataset
)

## Training: 
The next step after creating a model with an attached dataset is to train the model. KNN Classifiers are relatively quick to train, so we shouldn't have to wait long for this step to be complete. You can use the job ID returned from the following line of code to check when the training is complete. 

In [6]:
app.models.train_model(model)

## Testing: 
Be aware that if you run this next block before the model has completed training, you will recieve a "BoonSdkInvalidRequestException". Read more [here](https://app.gitbook.com/@zorroa/s/boonsdk/training-models/models/training-a-model) about how you can wait for the job to complete prior to analysis. Now we'll test the model we've trained by providing the url for a single image we would like to classify as either a resume or a cover letter. The result we print is a JSON object with one prediction comprised of a label and a confidence score. If the predicted label returned is 'Unrecognized', that means the model believes what you have provided is not a resume or a cover letter. 
**Note: this does not upload a new asset, it only analyzed an image you have referred to by url.**

In [10]:

url = "https://storage.googleapis.com/zorroa-public/datasets/docs_demo/test/test_resume1.jpg"

result = app.assets.analyze_file(url, ["document-classifier"])
print(result.get_analysis("document-classifier"))

BoonSdkInvalidRequestException: <BoonSdkRequestException msg=Your HTTP request was invalid '500', response not JSON formatted. Expecting value: line 1 column 1 (char 0)>

## Improving Performance
There are many different styles of resumes and and you might eventually run into a case where your model fails to recognize the correct type of document.  When this happens you can improve your model's performance by labeling more examples and retraining your model.